In [ ]:
# Download training data

import gdown

url = "https://drive.google.com/file/d/1VC9i5gvZAxCE-RkydXHdanXohY6OGO5P/view?usp=sharing"
output = "data/scrna/schiebinger.npz"
gdown.download(url, output, quiet=False, fuzzy=True)

In [ ]:
import sys, os
sys.path.append(os.path.join(sys.path[0], "code"))

In [ ]:
import pytorch_models as models
import pytorch_samplers as samplers
import pytorch_losses as losses
import pytorch_training as training
import pytorch_utils as utils
import sinkhorn_cnf as cnf
import scrna_exper as scrna
from geomloss import SamplesLoss
from torchdiffeq import odeint as odeint

import importlib
import time
import json
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import torch
import random
import numpy as np

importlib.reload(models)
importlib.reload(losses)
importlib.reload(samplers)
importlib.reload(training)
importlib.reload(utils)
importlib.reload(scrna)

In [ ]:
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Import WOT data

data_file = "data/scrna/schiebinger.npz"
data_dict = np.load(data_file, allow_pickle=True)

all_data = data_dict["original_embedding"]
# Rescale the data by factor of 1e-3
all_data *= 1e-3

timestamps = data_dict["sample_labels"]

times = np.unique(timestamps)

subsampled_data_list = []
data_list = []
# Generate training data by randomly selecting 500 observations per time point
for t in list(times):
    data_t = all_data[timestamps == t]
    data_list.append(data_t)
    np.random.shuffle(data_t) # shuffle rows of data_t
    subsampled_data_t = data_t[:500] # keep only first 500 samples
    subsampled_data_list.append(subsampled_data_t)

In [ ]:
# Learn metric tensor A(x) from Schiebinger data

seed = 0
n_samples = 500
space_dims = 2
scalar_hidden_dims = 128
matrix_hidden_dims = 2048
gp_strength_phi = 0
gp_strength_R = 1e1
fro_reg_strength = 1e0
identity_reg_strength = 0
lr = 1e-1
weight_decay = 1.5e-2
n_epochs_phi = 100
n_epochs_R = 1000
R_model_type = "singlelayer"

t0 = time.time()
scrna_learnedA, phi_list, rho_0_list, rho_1_list = training.twostep_train_wot_model(subsampled_data_list,
                                                                                    n_samples,
                                                                                    scalar_hidden_dims,
                                                                                    matrix_hidden_dims,
                                                                                    fro_reg_strength,
                                                                                    identity_reg_strength,
                                                                                    gp_strength_phi,
                                                                                    gp_strength_R,
                                                                                    lr,
                                                                                    weight_decay,
                                                                                    n_epochs_phi,
                                                                                    n_epochs_R,
                                                                                    seed,
                                                                                    R_model_type
                                                                                   )

t1 = time.time()
print('elapsed time: ' + str(t1-t0) + ' s.')

# Plot eigenvectors of metric
plt.figure(figsize=(20,20))
# Eigs plot params
x_lims = (-15,15)
y_lims = (-10,15)
n = 30
utils.eigs_quiver(scrna_learnedA, n, x_lims, y_lims)
for t in range(len(times)):
    label = "t = " + str(times[t])
    plt.scatter(subsampled_data_list[t][:,0], subsampled_data_list[t][:,1], label=label, alpha=0.2)

In [ ]:
# Save model

scrna_learned_path = "trained_models/scrna_learned_params.pt"
torch.save(scrna_learnedA.state_dict(), scrna_learned_path)

In [ ]:
# Load pretrained model

space_dims = 2
matrix_hidden_dims = 2048
A_fname = "trained_models/scrna_pretrained_params.pt"
scrna_learnedA = models.PSDMatrix(space_dims, matrix_hidden_dims).to(device)
scrna_learnedA.load_state_dict(torch.load(A_fname))

In [ ]:
# Generate list of samplers for all time points

rho_list = samplers.generate_all_eb_samplers(data_list)

In [ ]:
# Run experiments without A

k_vals = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
A_model = scrna_learnedA
for k in k_vals:
    json_fname = "results/scrna_experiments/k" + str(k) + "_noA.json"
    use_A = False
    W1_vals = scrna.run_experiment(rho_list, json_fname, k, A_model, use_A)
    json_fname_final_tps = "results/scrna_experiments/k" + str(k) + "_noA_final_tps.json"
    W1_vals_final_tps = scrna.run_final_tp_experiment(rho_list, json_fname_final_tps, k, A_model, use_A)

In [ ]:
# Run experiments with A

k_vals = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
A_model = scrna_learnedA
for k in k_vals:
    json_fname = "results/scrna_experiments/k" + str(k) + "_withA_lambd_1e1.json"
    use_A = True
    W1_vals = scrna.run_experiment(rho_list, json_fname, k, A_model, use_A)
    json_fname_final_tps = "results/scrna_experiments/k" + str(k) + "_withA_lambd_1e1_final_tps.json"
    W1_vals_final_tps = scrna.run_final_tp_experiment(rho_list, json_fname_final_tps, k, A_model, use_A)

In [ ]:
# CELLS BELOW ARE FOR GENERATING PLOTS FROM EXPERIMENTAL RESULTS

In [ ]:
# Compute W1 vals across multiple runs with metric tensor

k_vals = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
withA_mean_list = []
withA_std_list = []
use_A = True
n_runs = 10
for k in k_vals:
    mean, std = scrna.compute_multiple_runs(rho_list, k, use_A, n_runs)
    withA_mean_list.append(mean)
    withA_std_list.append(std)

In [ ]:
# Compute W1 vals across multiple runs without metric tensor

k_vals = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
noA_mean_list = []
noA_std_list = []
use_A = False
n_runs = 10
for k in k_vals:
    mean, std = scrna.compute_multiple_runs(rho_list, k, use_A, n_runs)
    noA_mean_list.append(mean)
    noA_std_list.append(std)

In [ ]:
# Generate scRNA W1 comparison (with vs without learned metric tensor)

plt.figure(figsize=(20,20))
plt.errorbar(k_vals, noA_mean_list, yerr = noA_std_list, label="Without A", linewidth=5, ecolor="black")
plt.errorbar(k_vals, withA_mean_list, yerr = withA_std_list, label="With A", linewidth=5, ecolor="black")
plt.xlabel("k", size=30)
plt.xticks(k_vals[::2])
plt.ylabel("EMD", size=30)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.legend(fontsize=30)
# Save figure
plt.savefig("results/scrna_emd_plot.png", dpi=300, bbox_inches="tight")

In [ ]:
# Generate scRNA trajectory inference plots

fig = plt.figure(figsize=(30,20))
ax1 = fig.add_subplot(2,3,1)
ax2 = fig.add_subplot(2,3,2)
ax4 = fig.add_subplot(2,3,4)
ax5 = fig.add_subplot(2,3,5)
ax3 = fig.add_subplot(2,3,3)
ax6 = fig.add_subplot(2,3,6)

# Do k=3 for case where A doesn't help much

k = 3
scrna.visualize_advected_samples_all_times(ax1, rho_list, k, use_A=False)
scrna.visualize_advected_samples_all_times(ax2, rho_list, k, use_A=True)

# Do k=15 for case where A helps a lot

k = 15
scrna.visualize_advected_samples_all_times(ax4, rho_list, k, use_A=False)
scrna.visualize_advected_samples_all_times(ax5, rho_list, k, use_A=True)

# Plot ground truth

norm = colors.Normalize(vmin=0, vmax=len(data_list))
cmap = cm.viridis
for t in range(len(data_list)):
    data = data_list[t][:1000]
    ax3.scatter(data[:,0], data[:,1], color=cmap(norm(t)), s=0.1)
    ax6.scatter(data[:,0], data[:,1], color=cmap(norm(t)), s=0.1)

ax1.set_title("Baseline", fontsize="xx-large", fontweight="bold")
ax2.set_title("Ours", fontsize="xx-large", fontweight="bold")
ax3.set_title("Ground truth", fontsize="xx-large", fontweight="bold")

# Save figure
plt.savefig("results/scrna_traj_plots.png", dpi=300, bbox_inches="tight")